In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import os
import cv2

import sys
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
import tensorflow as tf
import numpy as np


try:
    from labinet.io_util import load_label_map
    from labinet.io_util import load_model
    import labinet.object_detect
    import labinet.box
except ImportError:
    # This part is only required to run the notebook
    # directory when the module itself is not installed.
    #
    # If you have the module installed, just use "import labinet..."
    import os
    import inspect
    # the .travis.yml is coded so that we execute tests from within test subdir. Relative to 'test' the .py is found in ../source/dev
    cmd_subfolder = os.path.realpath(os.path.abspath(os.path.join(os.path.split(inspect.getfile(inspect.currentframe()))[0], "..")))
    if cmd_subfolder not in sys.path:
        sys.path.insert(0, cmd_subfolder)
    from labinet.io_util import load_label_map
    from labinet.io_util import load_model
    import labinet.object_detect
    import labinet.box

%aimport labinet.io_util
%aimport labinet.object_detect
%aimport labinet.box

%matplotlib inline

In [3]:
video = cv2.VideoCapture(0)

In [ ]:
# Try capturing one frame
ret, frame = video.read()
if frame is None:
    print("Error - did you connect your webcam?")
else:
    cv2.imshow('object detection', cv2.resize(frame, (640,480)))
    key_pressed = cv2.waitKey(25)
    print(key_pressed)    

In [18]:
# close the window again
cv2.destroyAllWindows()
video.release()

 ## Object Detection via Webcam
 Actually it is a loop to capture picture via webcam and to Object detection
 

In [4]:
# Paths to model etc pp
MODEL_NAME = 'inference_graph'   # the result from Step 6 Export Inference Graph
# Size, in inches, of the output images.
IMAGE_SIZE = (480, 640)
CWD_PATH = os.path.join(os.getcwd(),'..')  # should become gitbase
NUM_CLASSES = 1
# model
PATH_TO_MODEL = os.path.join(CWD_PATH, MODEL_NAME, 'frozen_inference_graph.pb')
# label map
LABEL_MAP = os.path.join(CWD_PATH, 'data', 'object-detection.pbtxt')


In [5]:
# load model and labels
detection_graph = load_model(PATH_TO_MODEL)
categories, category_index = load_label_map(LABEL_MAP, NUM_CLASSES)

In [6]:
# prepare tensor dict for inference
tensor_dict = labinet.object_detect.get_tensor_dict_with_masks(IMAGE_SIZE[1], IMAGE_SIZE[0], detection_graph)
image_tensor = tensor_dict['image_tensor']
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

set: num_detections:0
set: detection_boxes:0
set: detection_scores:0
set: detection_classes:0
set: image_tensor:0
tensor:0 for image.shape=(x=640,y=480)


In [32]:
# Prepare the Cam!
video = cv2.VideoCapture(0)
ret, frame = video.read()
if frame is None:
    print("Error - did you connect your webcam?")

In [7]:
%aimport labinet.object_detect
import cv2

In [ ]:
with tf.Session(graph=detection_graph, config=config) as sess:
    # capture
    while(True):
        ret, frame = video.read()
        #print(f"Captured frame.shape={frame.shape} - type(frame)={type(frame)}")
        image_np_exp = np.expand_dims(frame, axis=0) 
        #print(f'np-frame.shape={image_np_exp.shape}')
        # inference
        output_dict = sess.run(tensor_dict, feed_dict={image_tensor: image_np_exp})
        labinet.object_detect.convert_output_dict(output_dict)
        # visualize boxes
        image_with_boxes = labinet.object_detect.visualize_boxes_after_detection(frame, output_dict, category_index)
        # show image
        cv2.imshow('Detection Running...', cv2.resize(image_with_boxes,(IMAGE_SIZE[1],IMAGE_SIZE[0])))
        #cv2.waitKey(25)
        if cv2.waitKey(25) == ord('q'):
            break               
    


In [35]:
video.release()
cv2.destroyAllWindows()